# PROYECTO 3 SCRAPPING

Dime tu nombre y te dire quien eres
-Sacar de wikipedia el origen del nombre y el significado
- poblacion por codigo postal/ padron
- criminalidad por poblacion/
- politica
- numero de clinicas en ese codigo postal en google maps?¿
- edad media
- http://www.idescat.cat/poblacioestrangera/?b=10&geo=mun:080193&lang=es datos de inmigracion y del padron catalan
- educacion en cataluña: https://www.idescat.cat/pub/?q=08029&id=eep&lang=es#Plegable=geo
- API idescat https://www.idescat.cat/dev/api/emex/

Hecer un pseudocodigo?

## Importamos bibliotecas

In [259]:
import requests
import pandas as pd
import re
from bs4 import BeautifulSoup

In [463]:
#api de codigo postal
class CodigoPostal():
    def __init__(self,cp):
        #Podria poner un input
        self.cp=cp
        
    #informacion generalcon una api    
    def infor(self):
        url=f"http://api.zippopotam.us/ES/{self.cp}"
        resp=requests.get(url).json()
        df=pd.DataFrame(resp["places"])
        return df

    def renta(self):
        for c in range(len(df.Codigo_postal)):
            if self.cp==df.Codigo_postal.loc[c]:
                renta=df["renta_media_disponible"].loc[c]
                return(renta)
            elif self.cp[:4]==df.Codigo_postal.loc[c]:
                renta=df["renta_media_disponible"].loc[c]
                return(renta)
            elif self.cp[:3]==df.Codigo_postal.loc[c]:
                renta=df["renta_media_disponible"].loc[c]
                return(renta)
            elif self.cp[:2]==df.Codigo_postal.loc[c]:
                renta=df["renta_media_disponible"].loc[c]
                return(renta)
       
    def estudios_superiores(self):
        #municipio tiene que ser igual al nombre de infor y es solo para cataluña
        id_mun=id_cat.id[id_cat.mun==municipio].item()
        url=f"https://www.idescat.cat/pub/?geo=mun%3A{id_mun}&id=eep&n=14468&fil=3&col=3&lang=es#Plegable=geo"
        resp=requests.get(url)
        content=BeautifulSoup(resp.content,"html.parser")
        estudios_superiores=content.findAll("td",attrs={"data-title":"Valor"})[0].text
        return estudios_superiores
    
    def criminalidad(self):
        pass
    
    def polucion(self):
        pass
    
    def calculo_distancia(self):
        pass
    
    

In [630]:
cod=CodigoPostal(cp="08029")

In [631]:
cod.infor()

,place name,longitude,state,state abbreviation,latitude
0,Barcelona,2.1482,Cataluna,CT,41.386


# Repaso de clases y api simples

In [114]:
#api de edades por nombre
class Persona():
    def __init__(self,name=None):
        self.name=input("Como te llamas?")
        
    def edad(self):
        url=f"https://api.agify.io?name={self.name}" 
        resp=requests.get(url).json()
        if type(resp["age"])==int:
            return resp["age"]
        else:
            return "no te inventes nombres"

    #api de genero por nombre   
    def genero(self):
        url=f"https://api.genderize.io/?name={self.name}"
        resp=requests.get(url).json()
        if (resp["gender"])=="male":
            return "masculino"
        elif (resp["gender"]=="female"):
            return "femenino"
        else:
            return("no te inventes nombres")

    #api de nacionalidades por nombre
    def nacionalidad(self):
        url=f"https://api.nationalize.io/?name={self.name}" 
        resp=requests.get(url).json() 
        df=pd.DataFrame(data=resp["country"])
        df.columns=["Pais","Probabilidad"]
        return df
    

    #Aficion favorita: Boredapi
    def aficion(self):
        url=f"http://www.boredapi.com/api/activity?type=recreational"
        resp=requests.get(url).json()
        return resp["activity"]

    #Funcion princial
    def conocer(self):
        if edad(self.name)=="no te inventes nombres":
            return("Nombre no reconocido, intentalo de nuevo")
        else:
            datos={
                "Nombre":self.name,
                "Edad":edad(self.name),
                "Genero": genero(self.name),
                "Aficion": aficion(),
                    }
            df=pd.DataFrame.from_dict(datos,orient="index",columns=["Persona"])


            print(df)
            print("(---------------------)")
            print(nacionalidad(self.name))

In [115]:
persona=Persona()
persona.conocer()

Como te llamas?Estanislao
                              Persona
Nombre                     Estanislao
Edad                               57
Genero                      masculino
Aficion  Think of a new business idea
(---------------------)
  Pais  Probabilidad
0   ES             1


# Obtencion de datos con el codigo postal

- Datos a obtener a traves del cp:
    - Ciudad
    - longitud y latitud
    - provincia
    - renta per capita
- VAlidacion: Tienen que ser 5 numeros y existir

### Api basica de codigo postal

Direccion web: http://api.zippopotam.us/

In [119]:
#ejemplo con un codigo postal de barcelona e imprimo las llaves del json 
url=f"http://api.zippopotam.us/ES/08029"
resp=requests.get(url).json()
resp.keys()

dict_keys(['post code', 'country', 'country abbreviation', 'places'])

In [122]:
#imprimio las respuestas para ver cual me interesa mas, claramente es resp[places]
print(resp["post code"])
print(resp["country"])
print(resp["country abbreviation"])
print(resp["places"])

08029
Spain
ES
[{'place name': 'Barcelona', 'longitude': '2.1482', 'state': 'Cataluna', 'state abbreviation': 'CT', 'latitude': '41.386'}]


In [123]:
#funcion basica para esta api     
def infor(cp):
    url=f"http://api.zippopotam.us/ES/{cp}"
    resp=requests.get(url).json()
    df=pd.DataFrame(resp["places"])
    return df
#ejemplo
infor("08029")

,place name,longitude,state,state abbreviation,latitude
0,Barcelona,2.1482,Cataluna,CT,41.386


### scrapeando hacienda

Datos de la renta per capita:
"https://www.agenciatributaria.es/AEAT/Contenidos_Comunes/La_Agencia_Tributaria/Estadisticas/Publicaciones/sites/irpfCodPostal/2019/jrubikf44a5ab7a1e14e3ab549262b8a16cecff3a2b06c2.html"

Poner fotos de la tabla

# scrapping agencia tributaria- renta per capita por codigo postal

In [124]:
#url correcta cotilleando el inspector
url="https://www.agenciatributaria.es/AEAT/Contenidos_Comunes/La_Agencia_Tributaria/Estadisticas/Publicaciones/sites/irpfCodPostal/2019/jrubikf44a5ab7a1e14e3ab549262b8a16cecff3a2b06c2.html"
resps=requests.get(url)
content=BeautifulSoup(resps.content,"html.parser")

In [263]:
#Observamos la tabla
content.findAll("table")

[<table id="table01"><thead><tr>
 <th class="heading-heading" colspan="1" id="table01_cornerR0C0" rowspan="1" style="text-align:left;"><a href="javascript:void(0);" onclick="showHideTableRecursiveID('table03');return false;"><img alt="se expande la fila" height="13" id="imgID" src="Temas/comun/img/table/expand.gif" width="13"/></a> </th>
 <th class="heading-heading" id="table01_c1"><div>Número de</div>declaraciones</th>
 <th class="heading-heading" id="table01_c2"><div>Renta bruta</div>media</th>
 <th class="heading-heading" id="table01_c3"><div>Renta</div><div>disponible</div>media</th>
 <th class="heading-heading" id="table01_c4"><div>Rentas</div> del trabajo</th>
 <th class="heading-heading" id="table01_c5"><div>Rentas</div>exentas</th>
 <th class="heading-heading" id="table01_c6"><div>Renta</div>bruta</th>
 <th class="heading-heading" id="table01_c7"><div>Cotizaciones</div><div>sociales</div>a la SS</th>
 <th class="heading-heading" id="table01_c8"><div>Cuota </div><div>resultante 

In [142]:
#observo el contenido de las tr
print(content.findAll("tr")[0].text)# la primera son las columnas
print(content.findAll("tr")[1].text)


 
Número dedeclaraciones
Renta brutamedia
Rentadisponiblemedia
Rentas del trabajo
Rentasexentas
Rentabruta
Cotizacionessocialesa la SS
Cuota resultante deautoliquidación
Rentadisponible

 Almería-0401391.54425.68221.1361.908.018.94982.912.6342.351.070.75691.096.345325.105.9551.934.897.661


In [136]:
#creo una lista de todas las filas a partir de la segunda 
lista=content.findAll("tr")[1:]

In [132]:
# Para sacar el codigo postal y el nombre esta es la busqueda, ejemplo
content.findAll("tr")[2].find("th").text

'04001-Centro Rambla- Centro Paseo- Plaza de Toros- Catedral '

In [135]:
#esto seria para sacar la renta, ejemplo
content.findAll("tr")[2].find("td",attrs={"headers":"table01_c3"}).text

'36.352'

In [139]:
datos=[]
for cp in lista:
    dic={
        "nombre":cp.find("th").text,
        "renta_media_disponible": cp.find("td",attrs={"headers":"table01_c3"}).text
    }
    datos.append(dic)

In [171]:
df=pd.DataFrame(datos)
df

,nombre,renta_media_disponible
0,Almería-04013,21.136
1,04001-Centro Rambla- Centro Paseo- Plaza de To...,36.352
2,04002-La Chanca- Las Cuevas de S.Joaquín- Urb....,17.690
3,04003-Pza.de la Constitución- Cerro San Cristo...,20.935
4,04004-Oliveros- Santa Rita,26.800
...,...,...
681,26006-Varea- La Portalada,23.466
682,26007-La Cava-Fardachón-Las Gaunas-Siete Infan...,26.713
683,26008-San Adrián- La Cava,26.425
684,26009-El Campillo- Polígono Cantabria,22.568


###  limpieza del df

Problemas:
 - Valor de la renta no numerico
 - valores iguales en columna nombre

#### Cambio a valor numerico

In [172]:
#un poco de informacion de la tabla, no hay nulos pero la renta no es formato int
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 686 entries, 0 to 685
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   nombre                  686 non-null    object
 1   renta_media_disponible  686 non-null    object
dtypes: object(2)
memory usage: 10.8+ KB


In [173]:
#cambiamos el Dtype a valor int
df["renta_media_disponible"]=df["renta_media_disponible"].astype(float)

In [174]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 686 entries, 0 to 685
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   nombre                  686 non-null    object 
 1   renta_media_disponible  686 non-null    float64
dtypes: float64(1), object(1)
memory usage: 10.8+ KB


#### Valores iguales en columna nombre

In [163]:
#hay 40 provincias y cada provincia tiene agrupados los municipios que no salen en resto
df["nombre"].value_counts()

Resto                                            40
29007-Carranque-Hazas Cuevas                      1
46035-Sant Pau-Benimanet                          1
50016-Santa Isabel                                1
14001-Ollerías-Sta Marina-San Lorenzo             1
                                                 ..
35002-Triana                                      1
 Sabadell-08187                                   1
41009-Casco Antiguo-Macarena-Norte                1
41016-Bellavista-La Palmera-Cerro-Amate-Este      1
38107-El Sobradillo-La Gallega-El Pilar           1
Name: nombre, Length: 647, dtype: int64

In [170]:
df[df.nombre=="Resto "]

,nombre,renta_media_disponible
13,Resto,16.0
26,Resto,15.0
46,Resto,19.0
63,Resto,22.0
85,Resto,29.0
106,Resto,19.0
136,Resto,23.0
155,Resto,23.0
172,Resto,23.0
194,Resto,23.0


In [ ]:
FOTO limpieza resto

In [164]:
#Creo una lista con los depth_0 que son las capitales: 
municipio_principal=[]
lista2=content.findAll("tr",attrs={"class":"depth_0"})
for i in lista2:
    municipio_principal.append(i.find("th").text)

In [165]:
#Conveirto los nombres de resto
resto=[]
for i in municipio_principal:
    resto.append("resto "+i[1:-6])
resto

['resto Almería',
 'resto Jerez de la Frontera',
 'resto Córdoba',
 'resto Granada',
 'resto Málaga',
 'resto Sevilla',
 'resto Zaragoza',
 'resto Gijón',
 'resto Oviedo',
 'resto Palma de Mallorca',
 'resto Palmas de Gran Canaria Las',
 'resto Santa Cruz de Tenerife',
 'resto Santander',
 'resto Burgos',
 'resto Valladolid',
 'resto Albacete',
 'resto Badalona',
 'resto Barcelona',
 'resto Hospitalet de Llobregat',
 'resto Sabadell',
 'resto Sant Cugat del Vallès',
 'resto Terrassa',
 'resto Alicante/Alacant',
 'resto Elche/Elx',
 'resto Castellón de la Plana/Castelló de la Plana',
 'resto Valencia',
 'resto Coruña A',
 'resto Vigo',
 'resto Alcalá de Henares',
 'resto Alcobendas',
 'resto Alcorcón',
 'resto Getafe',
 'resto Leganés',
 'resto Madrid',
 'resto Móstoles',
 'resto Pozuelo de Alarcón',
 'resto Las Rozas',
 'resto Cartagena',
 'resto Murcia',
 'resto Logroño']

In [177]:
#Los sustituyo en el df
contador=0
for i in range(len(df.nombre)):
    if df.nombre[i]=="Resto ":
        df.nombre.loc[i]=resto[contador]
        contador+=1

C:\Users\irazu\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [179]:
#Compruebo
df.tail(3)

,nombre,renta_media_disponible
683,26008-San Adrián- La Cava,26.425
684,26009-El Campillo- Polígono Cantabria,22.568
685,resto Logroño,22.503


In [187]:
df.nombre.loc[7]

'04007-500 viviendas- El Zapillo- Cortijo Grande- Ciudad Jardín- La Bolera '

In [269]:
#Añado columna de codigo postal y quito el codigo de la columna nombre con regex
codigo_postal=[]
for i in range(len(df.nombre)):
    cp=(re.findall("[0-9]+",df.nombre.loc[i]))#saco todos los numeros de cada nombre
    try:
        codigo_postal.append(cp[0])#Como en algun nombre hay numeros que no corresponden al codigo postal soloc ojo el primero
        df.nombre.loc[i]=(re.sub(cp[0],"",df.nombre.loc[i])) #quito el cp
    except IndexError:
        codigo_postal.append(df.nombre.loc[i])
df["Codigo_postal"]=codigo_postal


C:\Users\irazu\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [296]:
df.head(15)

,nombre,renta_media_disponible,Codigo_postal
0,Almería,21.136,04013
1,Centro Rambla Centro Paseo Plaza de Toros ...,36.352,04001
2,La Chanca Las Cuevas de S.Joaquín Urb.Las P...,17.690,04002
3,Pza.de la Constitución Cerro San Cristobal B...,20.935,04003
4,Oliveros Santa Rita,26.800,04004
5,Barrio Alto Col.Montserrat Altamira,22.688,04005
6,Nueva Andalucía Regiones Devastadas El Diez...,21.973,04006
7,500 viviendas El Zapillo Cortijo Grande Ci...,23.786,04007
8,La Molineta Las Trincheras LosÁngeles La F...,17.041,04008
9,El Puche La Loma Las Chocillas Torre Cárde...,19.530,04009


In [272]:
#Quito los guiones
for i in range(len(df.nombre)):
    if re.findall("-",df.nombre.loc[i]):
        df.nombre.loc[i]=(re.sub("-"," ",df.nombre.loc[i]))

C:\Users\irazu\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [293]:
#Cambio el codigo de resto Ciudad por el inicio de su cp 
for i in range(len(df.Codigo_postal)):
    if df.Codigo_postal[i].isnumeric():
        continue
    else:
        print(df.Codigo_postal[i])

resto Almería
resto Jerez de la Frontera
resto Córdoba
resto Granada
resto Málaga
resto Sevilla
resto Zaragoza
resto Gijón
resto Oviedo
resto Palma de Mallorca
resto Palmas de Gran Canaria Las
resto Santa Cruz de Tenerife
resto Santander
resto Burgos
resto Valladolid
resto Albacete
resto Badalona
resto Barcelona
resto Hospitalet de Llobregat
resto Sabadell
resto Sant Cugat del Vallès
resto Terrassa
resto Alicante/Alacant
resto Elche/Elx
resto Castellón de la Plana/Castelló de la Plana
resto Valencia
resto Coruña A
resto Vigo
resto Alcalá de Henares
resto Alcobendas
resto Alcorcón
resto Getafe
resto Leganés
resto Madrid
resto Móstoles
resto Pozuelo de Alarcón
resto Las Rozas
resto Cartagena
resto Murcia
resto Logroño


In [298]:
#Cambios
dicc={
    "resto Almería": "04"   ,
    "resto Jerez de la Frontera": "11"   ,
    "resto Córdoba":  "14"  ,
    "resto Granada":   "18" ,
    "resto Málaga":   "29" ,
    "resto Sevilla":   "41" ,
    "resto Zaragoza": "50"   ,
    "resto Gijón":   "333" ,
    "resto Oviedo":  "331"  ,
    "resto Palma de Mallorca": "07"   ,
    "resto Palmas de Gran Canaria Las": "35"    ,
    "resto Santa Cruz de Tenerife":   "38" ,
    "resto Santander":  "39"  ,
    "resto Burgos":  "09"  ,
    "resto Valladolid": "47"   ,
    "resto Albacete":  "02",
    "resto Badalona":  "0891"  ,
    "resto Barcelona":   "080" ,
    "resto Hospitalet de Llobregat": "0890"   ,
    "resto Sabadell":  "0820",
    "resto Sant Cugat del Vallès": "0819"   ,
    "resto Terrassa":  "0822"  ,
    "resto Alicante/Alacant":  "036"  ,
    "resto Elche/Elx":   "033" ,
    "resto Castellón de la Plana/Castelló de la Plana":  "12",
    "resto Valencia": "46"   ,
    "resto Coruña A":  "15"  ,
    "resto Vigo":  "36"  ,
    "resto Alcalá de Henares":  "288" ,
    "resto Alcobendas": "281"   ,
    "resto Alcorcón": "2892"   ,
    "resto Getafe":  "2890"  ,
    "resto Leganés":  "2891"  ,
    "resto Madrid":   "280" ,
    "resto Móstoles":  "2893"  ,
    "resto Pozuelo de Alarcón": "2822"   ,
    "resto Las Rozas":  "2820"  ,
    "resto Cartagena": "305"   ,
    "resto Murcia":   "308" ,
    "resto Logroño":"26"
    
}

In [301]:
df.Codigo_postal.replace(dicc,inplace=True)

### funcion para obtener la renta media disponible por cp

Posible opcion de que no aparezca el codigo postal añadir la renta media:
df["renta_media_disponible"].sum()/len(df["renta_media_disponible"])

In [461]:
def renta(cp): 
    for c in range(len(df.Codigo_postal)):
        if cp==df.Codigo_postal.loc[c]:
            renta=df["renta_media_disponible"].loc[c]
            return(renta)
        elif cp[:4]==df.Codigo_postal.loc[c]:
            renta=df["renta_media_disponible"].loc[c]
            return(renta)
        elif cp[:3]==df.Codigo_postal.loc[c]:
            renta=df["renta_media_disponible"].loc[c]
            return(renta)
        elif cp[:2]==df.Codigo_postal.loc[c]:
            renta=df["renta_media_disponible"].loc[c]
            return(renta)
       


In [462]:
renta("08020")

22.984

###  APi idescat

In [490]:
#Desde esta url saco los id de cada pueblo
url="https://api.idescat.cat/emex/v1/geo.json"
resp=requests.get(url).json()

In [495]:
resp["fitxes"].keys()

dict_keys(['p', 'v', 'lang', 'version', 'o'])

In [520]:
resp["fitxes"]["v"]["v"]

{'scheme': 'com',
 'v': [{'scheme': 'mun', 'id': '080137', 'content': 'Avinyonet del Penedès'},
  {'scheme': 'mun', 'id': '080272', 'content': 'Cabanyes, les'},
  {'scheme': 'mun', 'id': '080581', 'content': 'Castellet i la Gornal'},
  {'scheme': 'mun', 'id': '080654', 'content': 'Castellví de la Marca'},
  {'scheme': 'mun', 'id': '080850', 'content': 'Font-rubí'},
  {'scheme': 'mun', 'id': '080919', 'content': 'Gelida'},
  {'scheme': 'mun', 'id': '080945', 'content': 'Granada, la'},
  {'scheme': 'mun', 'id': '081228', 'content': 'Mediona'},
  {'scheme': 'mun', 'id': '081458', 'content': 'Olèrdola'},
  {'scheme': 'mun', 'id': '081461', 'content': 'Olesa de Bonesvalls'},
  {'scheme': 'mun', 'id': '081542', 'content': 'Pacs del Penedès'},
  {'scheme': 'mun', 'id': '081640', 'content': 'Pla del Penedès, el'},
  {'scheme': 'mun', 'id': '081688', 'content': 'Pontons'},
  {'scheme': 'mun', 'id': '081748', 'content': 'Puigdàlber'},
  {'scheme': 'mun', 'id': '082068', 'content': 'Sant Cugat Se

In [521]:
fichas=resp["fitxes"]["v"]["v"]

In [527]:
id_cat=[]
for i in fichas:
    for n in i["v"]:
        dic={
            "mun":n["content"],
            "id":n["id"]
        }
        id_cat.append(dic)

In [530]:
id_cat=pd.DataFrame(id_cat)

### Sacamos el % de estudios superiores de ese municipio

In [532]:
id_mun="083054"
url=f"https://www.idescat.cat/pub/?geo=mun%3A{id_mun}&id=eep&n=14468&fil=3&col=3&lang=es#Plegable=geo"
resp=requests.get(url)
content=BeautifulSoup(resp.content,"html.parser")

In [538]:
estudios_superiores=content.findAll("td",attrs={"data-title":"Valor"})[0].text

### Funcion para sacar estudios superiores por municipio

In [623]:
#encontrar la forma de escribir bien el municipio, limpiar el df.mun
def estudios_superiores(municipio):
    id_mun=id_cat.id[id_cat.mun==municipio].item()
    url=f"https://www.idescat.cat/pub/?geo=mun%3A{id_mun}&id=eep&n=14468&fil=3&col=3&lang=es#Plegable=geo"
    resp=requests.get(url)
    content=BeautifulSoup(resp.content,"html.parser")
    estudios_superiores=content.findAll("td",attrs={"data-title":"Valor"})[0].text
    return estudios_superiores

In [624]:
estudios_superiores("Vilanova del Vallès")

'46,0'

In [629]:
estudios_superiores("Sitges")

'56,5'

In [618]:
id_cat[id_cat.mun=="Alcover"]["id"].item()

'430056'